In [155]:
import os
import glob
import pandas as pd

def concatenate(indir='/Users/vamsi_5/Downloads/updatedcsvs', outfile = '/Users/vamsi_5/Downloads/malwareCSV/concatenated.csv'):
    os.chdir(indir)
    fileList = glob.glob("*.csv")
    List=[]
    Colnames = ['IP Source','IP Destination','Source Port','Destination Port'
'TCP Flag Nonce',
'TCP Flag Congestion Window Reduced',
'TCP Flag ECN-Echo',
'TCP Flag Urgent',
'TCP Flag Acknowledgement',
'TCP Flag Push',
'TCP Flag Reset',
'TCP Flag Syn',
'TCP Flag Fin','Malicious']
    for i in fileList:
        print(i)
        df = pd.read_csv(i)
        List.append(df)
    concatdF = pd.concat(List,axis=0)
    
    return concatdF
    
    


In [156]:
Normal_DF = concatenate()

updated_2017-05-01_normal-2.pcap.csv
updated_2017-05-01_normal-3.pcap.csv
updated_2017-05-02_normal.pcap.csv
updated_2017-04-25_win-normal.pcap.csv
updated_2017_04_30-normal.pcap.csv
updated_2017-05-01_normal-4.pcap.csv
updated_2017-05-01_normal.pcap.csv
updated_2017-04-28_normal.pcap.csv
updated_2017-04-18_win-normal.pcap.csv
updated_2017-04-19_win-normal.pcap.csv


In [157]:
Normal_DF.head()

,IP Source,IP Destination,Source Port,Destination Port,TCP Flag Nonce,TCP Flag Congestion Window Reduced,TCP Flag ECN-Echo,TCP Flag Urgent,TCP Flag Acknowledgement,TCP Flag Push,TCP Flag Reset,TCP Flag Syn,TCP Flag Fin,Malicious
0,10.0.2.15,161.69.12.13,49157,80,0,0,0,0,0,0,0,1,0,0
1,161.69.12.13,10.0.2.15,80,49157,0,0,0,0,1,0,0,1,0,0
2,10.0.2.15,161.69.12.13,49157,80,0,0,0,0,1,0,0,0,0,0
3,10.0.2.15,161.69.12.13,49157,80,0,0,0,0,1,0,0,0,1,0
4,161.69.12.13,10.0.2.15,80,49157,0,0,0,0,1,0,0,0,0,0


In [158]:
Normal_DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6782934 entries, 0 to 253816
Data columns (total 14 columns):
IP Source                             object
IP Destination                        object
Source Port                           int64
Destination Port                      int64
TCP Flag Nonce                        int64
TCP Flag Congestion Window Reduced    int64
TCP Flag ECN-Echo                     int64
TCP Flag Urgent                       int64
TCP Flag Acknowledgement              int64
TCP Flag Push                         int64
TCP Flag Reset                        int64
TCP Flag Syn                          int64
TCP Flag Fin                          int64
 Malicious                            int64
dtypes: int64(12), object(2)
memory usage: 776.2+ MB


In [159]:
#Reading from multiple files, index re-starts at every new file. To have every thing in sequence
Normal_DF.reset_index(drop=True, inplace=True)

In [160]:
Normal_DF.columns

Index(['IP Source', 'IP Destination', 'Source Port', 'Destination Port',
       'TCP Flag Nonce', 'TCP Flag Congestion Window Reduced',
       'TCP Flag ECN-Echo', 'TCP Flag Urgent', 'TCP Flag Acknowledgement',
       'TCP Flag Push', 'TCP Flag Reset', 'TCP Flag Syn', 'TCP Flag Fin',
       ' Malicious'],
      dtype='object')

In [161]:
Normal_DF['IP Source']= Normal_DF['IP Source'].astype(str)
Normal_DF['IP Destination']= Normal_DF['IP Destination'].astype(str)


In [162]:
Normal_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6782934 entries, 0 to 6782933
Data columns (total 14 columns):
IP Source                             object
IP Destination                        object
Source Port                           int64
Destination Port                      int64
TCP Flag Nonce                        int64
TCP Flag Congestion Window Reduced    int64
TCP Flag ECN-Echo                     int64
TCP Flag Urgent                       int64
TCP Flag Acknowledgement              int64
TCP Flag Push                         int64
TCP Flag Reset                        int64
TCP Flag Syn                          int64
TCP Flag Fin                          int64
 Malicious                            int64
dtypes: int64(12), object(2)
memory usage: 724.5+ MB


In [163]:
#No of unique values are too much, Can we really label encode IP addresses ?
#Normal_DF['IP Source'].nunique()

8624

In [164]:
#Splitting the IP source into 4 columns based on '.'
Normal_DF['IP1'],Normal_DF['IP2'],Normal_DF['IP3'],Normal_DF['IP4'] = Normal_DF['IP Source'].str.split('.', 0).str

In [165]:
#Splitting the IP Destination into 4 columns based on '.'
Normal_DF['IP5'],Normal_DF['IP6'],Normal_DF['IP7'],Normal_DF['IP8'] = Normal_DF['IP Destination'].str.split('.', 0).str

In [166]:
Normal_DF.head()

,IP Source,IP Destination,Source Port,Destination Port,TCP Flag Nonce,TCP Flag Congestion Window Reduced,TCP Flag ECN-Echo,TCP Flag Urgent,TCP Flag Acknowledgement,TCP Flag Push,...,TCP Flag Fin,Malicious,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8
0,10.0.2.15,161.69.12.13,49157,80,0,0,0,0,0,0,...,0,0,10,0,2,15,161,69,12,13
1,161.69.12.13,10.0.2.15,80,49157,0,0,0,0,1,0,...,0,0,161,69,12,13,10,0,2,15
2,10.0.2.15,161.69.12.13,49157,80,0,0,0,0,1,0,...,0,0,10,0,2,15,161,69,12,13
3,10.0.2.15,161.69.12.13,49157,80,0,0,0,0,1,0,...,1,0,10,0,2,15,161,69,12,13
4,161.69.12.13,10.0.2.15,80,49157,0,0,0,0,1,0,...,0,0,161,69,12,13,10,0,2,15


In [168]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

In [169]:
#Label Encoding the columns from 14 to 22
X = Normal_DF.iloc[:, 14:22]

In [170]:
X.values[:,7]

array(['13', '15', '13', ..., '15', '156', '15'], dtype=object)

In [171]:
X.values[:,0] = encode.fit_transform(X.values[:,0])
X.values[:,1] = encode.fit_transform(X.values[:,1])
X.values[:,2] = encode.fit_transform(X.values[:,2])
X.values[:,3] = encode.fit_transform(X.values[:,3])
X.values[:,4] = encode.fit_transform(X.values[:,4])
X.values[:,5] = encode.fit_transform(X.values[:,5])
X.values[:,6] = encode.fit_transform(X.values[:,6])
X.values[:,7] = encode.fit_transform(X.values[:,7])


In [147]:
X['IP4'].nunique()

256

In [177]:
X.head()

,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8
0,1,0,112,57,61,221,24,35
1,61,221,24,35,1,0,112,57
2,1,0,112,57,61,221,24,35
3,1,0,112,57,61,221,24,35
4,61,221,24,35,1,0,112,57


In [ ]:
#Dropping the columns
Normal_DF.drop(['IP Source', 'IP Destination', 'IP1', 'IP2', 'IP3', 'IP4', 'IP5', 'IP6', 'IP7', 'IP8'], axis=1, inplace=True)

In [178]:
Normal_DF.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)

In [179]:
Normal_DF1 = Normal_DF + X

In [182]:
dfx = pd.concat([X,Normal_DF], axis =1) 
dfx.head()

,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8,Source Port,Destination Port,TCP Flag Nonce,TCP Flag Congestion Window Reduced,TCP Flag ECN-Echo,TCP Flag Urgent,TCP Flag Acknowledgement,TCP Flag Push,TCP Flag Reset,TCP Flag Syn,TCP Flag Fin,Malicious
0,1,0,112,57,61,221,24,35,49157,80,0,0,0,0,0,0,0,1,0,0
1,61,221,24,35,1,0,112,57,80,49157,0,0,0,0,1,0,0,1,0,0
2,1,0,112,57,61,221,24,35,49157,80,0,0,0,0,1,0,0,0,0,0
3,1,0,112,57,61,221,24,35,49157,80,0,0,0,0,1,0,0,0,1,0
4,61,221,24,35,1,0,112,57,80,49157,0,0,0,0,1,0,0,0,0,0


In [183]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6782934 entries, 0 to 6782933
Data columns (total 20 columns):
IP1                                   object
IP2                                   object
IP3                                   object
IP4                                   object
IP5                                   object
IP6                                   object
IP7                                   object
IP8                                   object
Source Port                           int64
Destination Port                      int64
TCP Flag Nonce                        int64
TCP Flag Congestion Window Reduced    int64
TCP Flag ECN-Echo                     int64
TCP Flag Urgent                       int64
TCP Flag Acknowledgement              int64
TCP Flag Push                         int64
TCP Flag Reset                        int64
TCP Flag Syn                          int64
TCP Flag Fin                          int64
 Malicious                            int64

In [184]:
dfx['Source Port'].nunique()

16324